## Failed Approach (Using API)

The following blocks try to use the API approach, which failed fantastically. I suggest you go to the next section, which works well.

You can also use directly PushishiftAPI() without psaw.

```Python
from pushshift_py import PushshiftAPI
import datetime as dt
import psaw
import pandas as pd
import requests
import json
import csv
import time
api = psaw.PushshiftAPI()

startEpoch = int(dt.datetime(2020,1,1).timestamp())
```
    
The following block shows how we can get information using pushshift. It shows how we can specify the features and get them. The returned data type is a generator with "submission" type as elements, though we can certainly make them into a list.

```Python
features = ['url','author', 'title', 'subreddit', 'id', 'created', 'score']
subreddit = 'NBA'

data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=10)

for datum in data:
    print(datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created, datum.score)

import praw

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="PASSWOR",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)
```

We can use praw to get the post body using the following block.
```Python
reddit.submission(id='eiev5d').selftext
```



In the following blocks, we create tables and store the information. For some reason, though, the api often acts up and freezes when we loop through the data.
```Python
import sqlite3

conn = sqlite3.connect('redditPosts.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Posts(
                id TEXT PRIMARY KEY,
                subreddit TEXT,
                title TEXT,
                author TEXT,
                url TEXT,
                created int)
                ''')

features = ['url','author', 'title', 'subreddit', 'id', 'created']
subreddit = 'stocks'
latest = dt.datetime(2021,5,8).timestamp()
earliest = dt.datetime(2020,1,1).timestamp()

startEpoch = earliest

while startEpoch <= latest:
    data = api.search_submissions(after=startEpoch,
                            subreddit=subreddit,
                            filter= features,
                            limit=100)
    
    for datum in data:
        print('Got here 2.')
        cur.execute('''INSERT OR IGNORE INTO Posts VALUES (?,?,?,?,?,?)'''
                    , (datum.id, datum.subreddit, datum.title, datum.author, datum.url, datum.created))
        
        currentTime = datum.created
    
    conn.commit()
    if currentTime == startEpoch:
        break
    startEpoch = currentTime + 1
    print(dt.datetime.fromtimestamp(startEpoch))    

```

## Another Approach (Getting JSON)

The method above is shaky at best. A lot of times the api just freezes. On the other hand, I find using requests much easier. The following code blocks contain what you need for storing reddit data you need.

In [1]:
import requests
import datetime as dt
import sqlite3
import json
import time
import sys

In [2]:
def getPushShiftData(after,before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?size=100&after='+str(int(after))+'&before='+str(int(before))+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']



def extractInfo(datum,features):
    info = {}
    
    for feature in features:
        info[feature] = datum[feature]
    
    return info

def getLatestTime(data):
    return data[-1]['created_utc']

def dataStoragePipeline(after, before, sub, conn):
    features = ['full_link','author', 'title', 'subreddit', 'id', 'created_utc', 'url']
    cursor = conn.cursor()
    while after < before:
        data = getPushShiftData(after, before, sub)
        if not data:
            print("There is no data anymore.")
            return 1
        for datum in data:
            cursor.execute('''INSERT OR IGNORE INTO Posts 
                                VALUES (?,?,?,?,?,?,?)'''
                              , (datum['id'], datum['subreddit'], datum['title'], datum['author'], datum['full_link'], datum['created_utc'], datum['url']))
        
        after = getLatestTime(data) + 1
        conn.commit()
        print("The latest post is submitted at", dt.datetime.fromtimestamp(after-1))
        time.sleep(0.1)
    
        
        

In [ ]:
cur.execute('''CREATE TABLE IF NOT EXISTS Posts(
                id TEXT PRIMARY KEY,
                subreddit TEXT,
                title TEXT,
                author TEXT,
                url TEXT,
                created int,
                ext_link TEXT)
                ''')

In [3]:
conn = sqlite3.connect('../redditPosts.sqlite')
cur = conn.cursor()
subreddit = 'GME'
end = int(time.time()-86400) #I subtracted by one day, so that we have some buffer.
start = dt.datetime(2021,1,1).timestamp()
cur.execute('''SELECT MIN(created), MAX(created) FROM Posts
                WHERE subreddit = ?''', (subreddit,))
datatimes = cur.fetchone()

if datatimes[0]:
    dataEarly, dataLate = datatimes
    if end < dataEarly:
        end = dataEarly
    elif start < dataLate:
        start =dataLate

In [4]:
while start < end:
    try:
        flag = dataStoragePipeline(after = start, before = end, sub = subreddit, conn = conn)
        if flag:
            break
    except KeyboardInterrupt:
        print("Interrupted by keyboard. Stopping.")
        break
        
    except:
        print("Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.")
        time.sleep(1)
        cur.execute('''SELECT MIN(created), MAX(created) FROM Posts
                        WHERE subreddit = ?''', (subreddit,))
        datatimes = cur.fetchone()
        
        if datatimes[0]:
            dataEarly, dataLate = datatimes
            if end < dataEarly:
                end = dataEarly
            elif start < dataLate:
                start =dataLate
        

The latest post is submitted at 2021-05-09 20:20:45
The latest post is submitted at 2021-05-10 04:28:47
The latest post is submitted at 2021-05-10 09:19:50
The latest post is submitted at 2021-05-10 11:54:26
The latest post is submitted at 2021-05-10 14:06:47
The latest post is submitted at 2021-05-10 16:53:46
The latest post is submitted at 2021-05-10 20:19:43
The latest post is submitted at 2021-05-11 03:16:43
The latest post is submitted at 2021-05-11 08:32:36
The latest post is submitted at 2021-05-11 11:00:37
The latest post is submitted at 2021-05-11 13:47:40
The latest post is submitted at 2021-05-11 16:16:36
The latest post is submitted at 2021-05-11 19:00:22
The latest post is submitted at 2021-05-11 21:43:02
The latest post is submitted at 2021-05-12 03:51:12
The latest post is submitted at 2021-05-12 08:39:21
The latest post is submitted at 2021-05-12 10:57:35
The latest post is submitted at 2021-05-12 12:45:01
The latest post is submitted at 2021-05-12 14:41:58
The latest p

In [ ]:
cur.execute('''SELECT subreddit,COUNT(*), COUNT(DISTINCT author) FROM Posts
                GROUP BY subreddit''')

print(cur.fetchall())

## Update the database by adding links.

```Python
cur.execute('''ALTER TABLE Posts
                ADD COLUMN ext_link TEXT''')

def dataUpdatePipeline(after, before, sub, conn):
    features = ['url', 'id']
    cursor = conn.cursor()
    while after < before:
        data = getPushShiftData(after, before, sub)
        if not data:
            print("There are no data anymore.")
            return 1
        for datum in data:
            cursor.execute('''UPDATE Posts
                                SET ext_link = ?
                                WHERE id = ? AND subreddit = ?'''
                              , (datum['url'], datum['id'], sub))
        
        after = getLatestTime(data) + 1
        conn.commit()
        print("The latest post is submitted at", dt.datetime.fromtimestamp(after-1))
        time.sleep(0.1)

def findStartingTime(cursor, subreddit):
    cursor.execute('''SELECT MAX(created) FROM Posts
                    WHERE subreddit = ? AND ext_link IS NOT NULL''', (subreddit,))
    datatimes = cursor.fetchone()
    
    return datatimes[0]

def getYourExistingSubs(cursor):
    cur.execute('''SELECT subreddit FROM Posts GROUP BY subreddit ORDER BY COUNT(*) ASC''')

    subreddits = list(map(lambda x: x[0], cur.fetchall()))
    
    return subreddits


#This chunk gives you all the subreddits you have scraped up to this point. Makes your life easier.
subreddits = getYourExistingSubs(cur)

for subreddit in subreddits:
    flag = 0
    print("Start working on {}".format(subreddit))
    end = int(time.time()-86400) #I subtracted by one day, so that we have some buffer.
    start = dt.datetime(2021,1,1).timestamp()

    dataTime = findStartingTime(cur,subreddit)
    if dataTime:
        start = max(start, dataTime)
    while True:
        try:
            dataUpdatePipeline(start, end, subreddit, conn)
            break
        except KeyboardInterrupt:
            print("Interrupted by keyboard. Stopping.")
            flag = 1
            break
        except:
            print("Error occurred. Probably due to frequent requests. Will resume working in 1 seconds.")
            dataTime = findStartingTime(cur,subreddit)
            if dataTime:
                start = max(start, dataTime)
            time.sleep(1)
    
    if flag:
        break
            
            

cur.execute('''SELECT subreddit FROM Posts GROUP BY subreddit ORDER BY COUNT(*) ASC''')

subreddits = list(map(lambda x: x[0], cur.fetchall()))

subreddits

```

# Using PRAW to Download Additional Features

In this section, we will use praw to scrap post body, score, and upvote_ratio.

**Please use your own client_id, secret, etc. since we might be scraping at the same time. I don't want to get this account locked up.**


In [4]:
import sqlite3
import praw
import time
import datetime as dt

conn = sqlite3.connect('../redditPosts.sqlite')
cur = conn.cursor()

reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="PASSWOR",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)


```Python
Sub = reddit.submission(id='eiev5d')
print(Sub.title, Sub.score, Sub.upvote_ratio)

#print(vars(Sub))

subreddit = 'options'
cur.execute('''SELECT id FROM Posts
                WHERE subreddit == ?
                LIMIT 10''', (subreddit,))

ids = cur.fetchall()


tStart = time.time()
for i,postId in enumerate(ids):
    Sub = reddit.submission(id=postId[0])
    print(Sub.selftext, Sub.score, Sub.upvote_ratio, Sub.url)
    

tEnd = time.time()

print(tEnd-tStart)


cur.execute('''CREATE TABLE IF NOT EXISTS PostBodyAndScore (
                    id TEXT PRIMARY KEY,
                    body TEXT,
                    score INT,
                    upvote_ratio FLOAT)''')

```

In [5]:
def timeLookupPRAW(cursor, subreddit):
    cursor.execute('''SELECT MIN(P.created), MAX(P.created)
                        FROM Posts P JOIN PostBodyAndScore B
                        ON P.id = B.id
                        WHERE subreddit = ?''', (subreddit,))
    return cursor.fetchone()

def retrieveID(cursor, num, subreddit, start, end):
    cursor.execute('''SELECT P.id
                        FROM (SELECT id, created FROM Posts
                                WHERE created >= ? AND created <= ? AND subreddit = ?) P LEFT JOIN PostBodyAndScore B
                        ON P.id = B.id
                        WHERE B.score IS NULL
                        ORDER BY P.created ASC
                        LIMIT ?''', (start,end, subreddit, num))
    
    return cursor.fetchall()

def storePostBodyAndScore(cursor, reddit, ids):
    flag = 0
    for i,postId in enumerate(ids):
        try:
            Sub = reddit.submission(id=postId[0])
            cursor.execute('''INSERT OR IGNORE INTO PostBodyAndScore
                                VALUES (?,?,?,?)''', (postId[0],Sub.selftext,Sub.score,Sub.upvote_ratio))
        except KeyboardInterrupt:
            flag = 1
            break
        except:
            print("Some error happened. Will abandon this post.")
            continue
        
    print("Finished adding post info for this batch.")
    conn.commit()
    return flag

def fillOutTable(cursor, reddit, subreddit, start, end, num = 500):
    delta = 86400
    
    _, latest = timeLookupPRAW(cursor,subreddit)
    
    if latest:
        start = max(start,latest)
    
    while start < end:
        ids = retrieveID(cursor, num, subreddit, start, start + delta)
        if not ids:
            start += delta
            print("==Finished all posts up to {}==".format(dt.datetime.fromtimestamp(start)))
            continue
        
        flag = storePostBodyAndScore(cursor, reddit, ids)
        if flag:
            return flag
            

    

In [6]:
#subreddits = getYourExistingSubs(cur)
subreddits = ['GME']
end = int(time.time()-86400)
start = dt.datetime(2021,1,1).timestamp()

for subreddit in subreddits:
    print("Currently working on subreddit {}".format(subreddit))
    flag = fillOutTable(cur, reddit, subreddit, start, end)
    if flag:
        break


Currently working on subreddit GME
==Finished all posts up to 2021-05-26 03:31:18==
